In [10]:
pip install deep-translator yandex-translater translate

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ------------- -------------------------- 1.3/4.0 MB 7.8 MB/s eta 0:00:01
   --------------------------------- ------ 3.4/4.0 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 8.0 MB/s  0:00:00

   -------------------- ------------------- 2/4 [click]
   ---------------------------------------- 4/4 [translate]

Note: you may need to restart the kernel to use updated packages.


In [31]:
from deep_translator import GoogleTranslator
from deep_translator import MyMemoryTranslator
import random
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from yandex.Translater import Translater
import requests
from translate import Translator

In [5]:
df = pd.read_csv('raw_data.csv') # изменить путь
df

,idx,text,label
0,10026,[PERSON]\r\n\r\nУважаемый [PERSON]!\r\n\r\n[OR...,Блок директора по проектированию
1,1005,[ORGANIZATION] инжиниринг общество с ограничен...,Блок деректора по газу
2,1010,[ORGANIZATION] ИНВЕСТ Общество с ограниченной ...,Блок заместителя генерального директора по без...
3,1013,[ORGANIZATION] филиал [OBJECT] имени [PERSON] ...,Блок технического директора
4,10186,[ORGANIZATION] ЗАМЕСТИТЕЛЬ ГЕНЕРАЛЬНОГО ДИРЕКТ...,Блок директора по проектированию
...,...,...,...
1416,7295,[ORGANIZATION] юридический адрес: [LOCATION] О...,Блок технического директора
1417,7296,[ORGANIZATION] сообщает [PERSON] о готовности ...,Блок технического директора
1418,7297,[ORGANIZATION] Юридический адрес: [LOCATION] О...,Блок технического директора
1419,27278G,ООО [НАЗВАНИЕ_КОМПАНИИ] Контактная информация:...,Проект «Трубопроводный транспорт Ещё одного НГКМ»


In [ ]:
blacklist = ['[PROJECT]', '[TYPE]', '[SIZE]', '[НОМЕР_ОБЪЕКТА]', '[REGIONAL_OFFICE]', '[NUMBER]', '[MUNICIPAL_DISTRICT]', '[OGRN]', '[CITY]', '[TIME]', '[DIMENSIONS]', '[NUMBER_OF_PAGES]', '[PRODUCT]', '[SIGNATURE]', '[SERVICE_AREA]', '[LAW_NUMBER]', '[QUANTITY]', '[PAGE]', '[PRODUCT_MARK]', '[PROJECT_NUMBER]', '[EXECUTIVE_DIRECTOR]', '[TEMPERATURE]', '[DOMAIN]', '[FILIAL]', '[DATE]', '[KPP]', '[VOLUME]', '[RESOURCE_TYPES]', '[SKVAGH_NUMBER]', '[DIAGNOSIS]', '[PAGE_NUMBER]', '[FRACTION]', '[WORK_TYPE]', '[ELECTRONIC_SIGNATURE]', '[DOCUMENT_SIGNATURE]', '[NETWORK]', '[NAME_OF_FIELD]', '[DISTANCE]', '[AUTONOMOUS_REGION]', '[URL]', '[DISTRICT]', '[DIRECTOR]', '[PLACE]', '[PLAN]', '[CHEMICAL_REAGENT]', '[WWW]', '[TITLE]', '[PROJECT_NAME]', '[PHONE_PROVIDER]', '[SYSTEM]', '[REGION]', '[ID]', '[LICENSE]', '[CURRENCY]', '[BIC]', '[PIPELINE]', '[AREA]', '[RIVER]', '[DEPOSITS]', '[ЛОКАЦИЯ]', '[ACTIVITY]', '[LENGTH]', '[DELIVERY_DATE]', '[BANK_ID]', '[YEAR]', '[JOB_TITLE]', '[TYPE_OF_ORDER]', '[WORK]', '[CERTIFICATE]', '[DEVICE]', '[STANDARD]', '[REPUBLIC]', '[SPEED]', '[НОМЕР_ДОКУМЕНТА]', '[ADDRESS]', '[ROOM]', '[PERIOD]', '[MODEL]', '[TDU]', '[MINISTRY]', '[BUILDING_NUMBER]', '[KPZ]', '[ORGANIZATION]', '[EVENT]', '[ORGANIZATION_ID]', '[STAGE]', '[POSITION]', '[COPY_COUNT]', '[PROCESSING_STANDARD]', '[EMAIL]', '[SNAB]', '[АДРЕС_ЭЛ_ПОЧТЫ]', '[DIVISION]', '[ACTION]', '[MARK]', '[INSTRUMENT_TYPE]', '[POSTAL_CODE]', '[STAND]', '[VALUE]', '[COPIES]', '[POSTCODE]', '[BANK]', '[STORAGE_PRICE]', '[OFFICE]', '[LETTER]', '[DEPARTMENT]', '[AMOUNT]', '[LAYER]', '[WIND_DIRECTION]', '[REGIONAL_AREA]', '[SPECIFICATION]', '[CATEGORY]', '[CONFERENCING]', '[PROJECT_CODE]', '[REGULATORY_INTERACTION]', '[WAREHOUSE_LIMIT]', '[OPERATOR]', '[PRICE_PER_UNIT]', '[TOPIC]', '[CODE]', '[UNIT]', '[WEBSITE]', '[OPERATION]', '[ДАТА]', '[SECTION]', '[TABLE_NUMBER]', '[COUNTRY]', '[VH_NO]', '[LOCATION]', '[CONNECTION_TYPE]', '[FINANCИAL_DATA]', '[DIMENSION]', '[SOFTWARE]', '[NOTES]', '[INTERNAL_REGULATION]', '[P]', '[ENGINE_MODEL]', '[PROTOCOL]', '[LICENSED_AREA]', '[INDIVIDUAL_PREMISE]', '[ИМЯ_ЧЕЛОВЕКА]', '[FUNCTION]', '[PROSPECT]', '[SCHEME]', '[GAS_PIPELINE]', '[STANDARD_NUMBER]', '[LAND_TYPE]', '[INN]', '[SERVICE_CODE]', '[DO]', '[COPY]', '[VEHICLE]', '[INN_KPP]', '[SUBORGANIZATION]', '[CONDITIONS]', '[REPRESENTATIVE]', '[NPS_AREA]', '[DOCUMENT_NUMBER]', '[PROFESSION]', '[BRANCH]', '[HOUSE]', '[CONFERENCING_LINK]', '[CENTRE]', '[BRAND]', '[ОБЛАСТЬ]', '[НОМЕР_ТЕЛЕФОНА]', '[NAME]', '[NOMINATION_2]', '[REGULATORY_DOCUMENT]', '[GEN_DIR]', '[ORGANIZATION_NUMBER]', '[PERSON]', '[ADMINISTRATION]', '[OBJECT]', '[GROUP]', '[OKPO]', '[THICKNESS]', '[ROUTE]', '[PAGE_COUNT]', '[PRODUCT_CODE]', '[PAGES]', '[DOCUMENT]', '[CONDITION]', '[VEHICLE_TYPE]', '[FIELD]', '[PHONE]', '[ETHNIC_GROUP]', '[MANAGER]', '[RECONSTRUCTION]', '[FINANCIAL_DIRECTOR]', '[НАЗВАНИЕ_КОМПАНИИ]', '[COMPETITION_NAME]', '[SERVICES]', '[MEASURES]', '[DESCRIPTION]', '[RESOLUTION]', '[DEPTH]', '[HALL]', '[FINANCИАL_DATA]', '[GOVERNMENT]', '[CENTRAL_BANK_OF_RUSSIA]', '[COPY_TO]', '[COMPRESSOR_MODEL]', '[ISLAND]', '[DIRECTION]', '[PRODUCT_TYPE]', '[MONTH]', '[GENERAL_DIRECTOR]', '[STATION]', '[POST_CODE]', '[WEB]', '[WORKPLACE]', '[OFFICE_NUMBER]', '[MAX_QUANTITY_ON_WAREHOUSE]', '[FINANCИАЛЬ_ДАННЫЕ]', '[QUARTER]', '[FINANCИАЛЬ_DATA]', '[FINANCIAL_DATA]', '[DATE_RANGE]', '[YEAR+1]', '[DATE_TIME]', '[DOCUMENT_TYPE]', '[FILE_EXTENSION]', '[ZIP_CODE]', '[CONTACT]', '[ВРЕМЯ]', '[FILE_SIZE]', '[MUNICIPAL_AREA]', '[INSTITUTION]', '[TAX_ID]', '[TOTAL_PRICE]']

LIMIT = 400

def translate(count: int, base_idx: str, base_text: str, label: str) -> list:
    results = []
    
    # Заменяем слова из блеклиста, чтобы переводчик их не менял
    for i in range(len(blacklist)):
        base_text = base_text.replace(blacklist[i], f'__{i}__')
    
    
    # Если длина текса больше LIMIT символов - разбиваем на куски
    if len(base_text) > LIMIT:
        sub_str_count = (len(base_text) + LIMIT - 1) // LIMIT
        print(sub_str_count)
        new_texts = ['' for _ in range(count)]
        exploded_text = [base_text[i:i + LIMIT] for i in range(0, len(base_text), LIMIT)]
        for i in range(sub_str_count-1):
            while exploded_text[i][-1] != ' ':
                exploded_text[i+1] = exploded_text[i][-1] + exploded_text[i+1]
                exploded_text[i] = exploded_text[i][:-1]
        for i in range(sub_str_count):
            new_sub_strs = [i['text'] for i in translate(count, base_idx, exploded_text[i], label)]
            for j in range(count):
                new_texts[j] += ' ' + new_sub_strs[j]
    
        for i in range(count):
            results.append({
                'idx': f'new_{label}_translate_{base_idx}_{i+1}',
                'text': new_texts[i],
                'label': label
            })
            
        return results
        
    # список языков для случайного перевода (можно менять)
    # languages = ['en', 'de', 'fr', 'es', 'it']
    languages = ['english', 'german', 'french', 'spanish', 'italian']
    for i in range(count):
        #new_text = '' + f" [перевод_{i+1}]" # base_text + f" [перевод_{i+1}]" # текст-рыба

        try:
            # выбираем случайный язык для “обратного перевода”
            lang1 = random.choice(languages)
            lang2 = random.choice(languages)

            # переводим с русского на lang1
            # text_foreign = GoogleTranslator(source='ru', target=lang1).translate(base_text)
            # print(123)
            # переводим с lang1 на lang2
            # new_text = GoogleTranslator(source=lang1, target='ru').translate(text_foreign)
            # print(123)    
            # и обратно на русский
            # new_text = GoogleTranslator(source=lang2, target='ru').translate(text_foreign)
            # print(123)
            # лёгкий sleep, чтобы не словить лимиты API
            # new_text = yt.translate('ru', lang1, base_text)
            # new_text = yt.translate(lang1, lang2, new_text)
            # new_text = yt.translate(lang2, 'ru', new_text)
            new_text = GoogleTranslator(source='russian', target=lang1).translate(base_text)
            
            new_text = GoogleTranslator(source=lang1, target=lang2).translate(base_text)
            
            new_text = GoogleTranslator(source=lang2, target='russian').translate(base_text)
            
            time.sleep(1)

        except Exception as e:
            print(base_text)
            print(f"⚠️ Ошибка перевода письма {base_idx}_{i+1}: {e}")
            # если не удалось — fallback: просто тег
            new_text = base_text + f" [перевод_{i+1}]"
            
        # Возвращаем слова из блеклиста
        for i in range(len(blacklist)):
            new_text = new_text.replace(f'__{i}__', blacklist[i])
        results.append({
            'idx': f'new_{label}_translate_{base_idx}_{i+1}',
            'text': new_text,
            'label': label
        })
    return results

In [54]:
# df[df['text'].str.len() > 10000]

# text = df[df['idx'] == '1215']['text'].values[0]
# text
text = '[ORGANIZATION] Адрес местонахождения: [LOCATION] Тел.: [CONTACT], e-mail: [CONTACT] Адрес местонахождения Обособленного подразделения No1: [LOCATION]; Адрес местонахождения Обособленного подразделения No2: [LOCATION]; Адрес местонахождения Обособленного подразделения No3: [LOCATION]. Тел.: [CONTACT], факс: [CONTACT], ОГРН [ID], ИНН [ID], www.[DOMAIN] Директору по капитальному строительству [ORGANIZATION] [PERSON] Копия: Руководителю проекта Трубопроводный транспорт Чаяндинского НГКМ [ORGANIZATION] [PERSON] [DATE_TIME] О согласовании стоимости МТР Уважаемый [PERSON]! В ответ на Ваши исх. No [DOCUMENT_NUMBER] от [DATE_TIME] и [DOCUMENT_NUMBER] от [DATE_TIME] О согласовании стоимости МТР до базиса поставки УПН Чаяндинского месторождения. В связи с изменением объема согласованной к закупу кабельно-проводниковой продукции, в том числе переноса большего объема в поставку Заказчика, прошу Вас направленные позиции в полном объеме письма NoОП2-715 от [DATE_TIME] и позиции с 1 по 3 ранее направленного письма NoОП2-711 от [DATE_TIME] считать не действительными. Ведомости с откорректированными объемами кабельно-проводниковой продукции будут направлены в ближайшее время. Дополнительно прошу Вас направить в наш адрес откорректированную разделительную ведомость по объекту: Обустройство Чаяндинского НГКМ. Куст газовых скважин No2. Приложение: 1. Письмо No [DOCUMENT_NUMBER] от [DATE_TIME], 2. Письмо No [DOCUMENT_NUMBER] от [DATE_TIME]. Исполнительный директор Исполнитель: [PERSON] сот.: [CONTACT] A.P. [PERSON] [ORGANIZATION] [ORGANIZATION] Юридический адрес: [LOCATION] ОГРН [ID], ИНН [ID]. Адрес для корреспонденции: [LOCATION] Тел.: [CONTACT] e-mail: [CONTACT] www.[DOMAIN] На No ОП2-715 от [DATE_TIME] О согласовании стоимости МТР Уважаемый [PERSON]! В ответ на Ваше письмо О согласовании стоимости МТР по объекту: Обустройство Чаяндинского НГКМ. Куст газовых скважин No2 информируем Вас, что Заказчиком проведен анализ стоимости МТР поставки Подрядчика, отсутствующих в составе УЕР и Прейскуранте МТР в рамках договора No [DOCUMENT_NUMBER] от [DATE_TIME] на выполнение строительно-монтажных работ. Информация по предельной цене закупки МТР с учетом доставки до базиса поставки указана в приложении к настоящему письму. Просим обеспечить закуп и поставку МТР на объект строительства по стоимости, не превышающей цены в приложении, при этом право выбора поставщика остается за подрядной организацией. Приложение: 1. Ведомость МТР [DOCUMENT_NUMBER] от [DATE_TIME] на 1 л. С уважением, Руководитель проекта Трубопроводный транспорт ЧНГКМ [PERSON] ДОКУМЕНТ ПОДПИСАН УСИЛЕННОЙ КАВАЛИФИЦИРОВАННОЙ ЭЛЕКТРОННОЙ ПОДПИСЬЮ Сертификат: [ID] Владелец: [PERSON] Действителен: с [DATE_TIME] по [DATE_TIME] [ORGANIZATION] Bx. No [DOCUMENT_NUMBER] от [DATE_TIME] Заказчик: [ORGANIZATION] Номер документа: [DOCUMENT_NUMBER] [ORGANIZATION] Доработ: No [DOCUMENT_NUMBER] от [DATE_TIME] Ведомость МТР [DOCUMENT_NUMBER] от [DATE_TIME] Организация: [ORGANIZATION] Объект: Обычный проект по ремонту и обслуживанию оборудования ХТКМ. Кустра30бхс кважин No2 No n/n Код МТР Наименование МТР (в соответствии с ECM) Единица измерения Стоимость материально-технических ресурсов (руб.) Общая стоимость (руб.) Дата поставки Поставщик  1 2 3 1 770001980569 Кабель Arctic Cab B3-P3БНhr(A)-HF-3ХЛ 34,0Мк-1ТУ [LOCATION]-004-19280226-2022 км [FINANCIAL_DATA] [DATE_TIME] [PERSON] 2 770001980559 Кабель Arctic Cab B3-P3НТhr(A)-HF-3ХЛ 56,0Мк ([CONTACT])-1ТУ [LOCATION]-004-19280226-2022 км [FINANCIAL_DATA] [DATE_TIME] [PERSON] 3 770001980538 Кабель Arctic Cab B3-P3НТhr(A)-HF-3ХЛ 41,5Мк (PE)-1ТУ [LOCATION]-004-19280226-2022 км [FINANCИАЛЬ_DATA] [DATE_TIME] [PERSON] 4 770001980516 Кабель Arctic Cab B3-P3НТhr(A)-HF-3ХЛ 21,5Мк (N)-1ТУ [LOCATION]-004-19280226-2022 км [FINANCIAL_DATA] [DATE_TIME] [PERSON] 5 770001980492 Кабель Arctic Cab INSTALL KMVBhr(A)-LS 51,0МВЭК-ХЛТУ 27.32.13-003-19280226-2019 км [FINANCIAL_DATA] [DATE_TIME] [PERSON] Снабжено: [PERSON] [PERSON] [PERSON] Наименование заказчика: [ORGANIZATION] Номер документа: [DOCUMENT_NUMBER] [ORGANIZATION] Юридический адрес: [LOCATION] ОРГН [ID], ИНН [ID]. Адрес для корреспонденции: [LOCATION] Тел.: [CONTACT] e-mail: [CONTACT] www.[DOMAIN] На No ОП2-711 от [DATE_TIME] Генеральному директору [ORGANIZATION] [PERSON] Уважаемый [PERSON]! В ответ на Ваше письмо О согласовании стоимости МТР по объекту: Транспорт газа на УППГ-4 Чаяндинского НГКМ. Газопровод информируем Вас, что Заказчиком проведен анализ стоимости МТР поставки Подрядчика, отсутствующих в составе УЕР и Прейскуранте МТР в рамках договора No [DOCUMENT_NUMBER] от [DATE_TIME] на выполнение строительно-монтажных работ. Информация по предельной цене закупки МТР с учетом доставки до базиса поставки указана в приложении к настоящему письму. Просим обеспечить закуп и поставку МТР на объект строительства по стоимости, не превышающей цены в приложении, при этом право выбора поставщика остается за подрядной организацией. Приложение: Ведомость МТР [DOCUMENT_NUMBER] от [DATE_TIME] г. на 1 л. С уважением, Руководитель проекта Трубопроводный транспорт Чаяндинского НГКМ [PERSON] [CONTACT] (78865) [CONTACT] [ORGANIZATION] Рег. No [DOCUMENT_NUMBER] от [DATE_TIME] Вх. No [DOCUMENT_NUMBER] от [DATE_TIME]'


translate(1, '1215', text, 'Имущественные вопросы')

12
2
2
2
2
2


[{'idx': 'new_Имущественные вопросы_translate_1215_1',
  'text': ' [ORGANIZATION] Адрес места нахождения: [LOCATION] Тел.: [CONTACT], e-mail: [CONTACT] Адрес №1: [LOCATION]; Адрес места нахождения Обособленного подразделения №2: [LOCATION]; Адрес места нахождения Обособленного подразделения №3: [LOCATION]. Тел.: [CONTACT], факты: [CONTACT], ОГРН [ID], ИНН [ID], www.[DOMAIN] Директор по капитальному строительству [ORGANIZATION] [PERSON] Копия:  Руководитель трубопроводного транспорта Чаяндинского НГКМ [ORGANIZATION] [PERSON] [DATE_TIME] О согласовании стоимости МТР Уважаемый [PERSON]! В ответ Вашим. № [DOCUMENT_NUMBER] от [DATE_TIME] и [DOCUMENT_NUMBER] от [DATE_TIME] О согласовании стоимости МТР до базиса поставки УПН Чаяндинского месторождения. В контексте измерения кабеля, подключенного к корпусу Изделие, у которого вал больше объем в  поставка Заказчика, прошу Вас направленные позиции в полном объеме писем №ОП2-715 от [DATE_TIME] и позиций с 1 п 3 ранее направленных писем №ОП2-711 о

In [16]:
a = 'faknksbdfdsv,msdvs.dgs.dfg'
# print([a[i:i + 5] for i in range(0, len(a), 5)])
a[:-1]

'faknksbdfdsv,msdvs.dgs.df'

In [29]:
# ajesbvk0hph27ishbpo9 - key id
# b1gt2mvdma9uqjh5o78p - folder id
# AQVN3F_oCOYiPLkV5wmlXCuZutXUQyhgZjWpv88Q
API_KEY = 'AQVN3F_oCOYiPLkV5wmlXCuZutXUQyhgZjWpv88Q'
folder_id = 'b1gt2mvdma9uqjh5o78p'

headers = {
    "Content-Type": "application/json",
    # Параметры для аутентификации с помощью API-ключа от имени сервисного аккаунта:
    "Authorization": f"Api-Key {API_KEY}",
}

target_language = "ru"
texts = ["Hello", "World"]

body = {
    "targetLanguageCode": target_language,
    "texts": texts,
    "folderId": folder_id,
}

response = requests.post(
    "https://translate.api.cloud.yandex.net/translate/v2/translate",
    json=body,
    headers=headers,
)

print(response.text)

{
 "code": 7,
 "message": "Permission to [resource-manager.folder b1gt2mvdma9uqjh5o78p, resource-manager.cloud b1geh4por0cktf0aueld, organization-manager.organization bpfb6jn2ubvusl03274b] denied",
 "details": [
  {
   "@type": "type.googleapis.com/google.rpc.RequestInfo",
   "requestId": "4234f0d7-af63-43ef-989f-e8e7232cceb1"
  }
 ]
}



In [37]:
text = '[ORGANIZATION] Адрес местонахождения: [LOCATION] Тел.: [CONTACT], e-mail: [CONTACT] Адрес местонахождения Обособленного подразделения No1: [LOCATION]; Адрес местонахождения Обособленного подразделения No2: [LOCATION]; Адрес местонахождения Обособленного подразделения No3: [LOCATION]. Тел.: [CONTACT], факс: [CONTACT], ОГРН [ID], ИНН [ID], www.[DOMAIN] Директору по капитальному строительству [ORGANIZATION] [PERSON] Копия: Руководителю проекта Трубопроводный транспорт Чаяндинского НГКМ [ORGANIZATION] [PERSON] [DATE_TIME] О согласовании стоимости МТР Уважаемый [PERSON]! В ответ на Ваши исх. No [DOCUMENT_NUMBER] от [DATE_TIME] и [DOCUMENT_NUMBER] от [DATE_TIME] О согласовании стоимости МТР до базиса поставки УПН Чаяндинского месторождения. В связи с изменением объема согласованной к закупу кабельно-проводниковой продукции, в том числе переноса большего объема в поставку Заказчика, прошу Вас направленные позиции в полном объеме письма NoОП2-715 от [DATE_TIME] и позиции с 1 по 3 ранее направленного письма NoОП2-711 от [DATE_TIME] считать не действительными. Ведомости с откорректированными объемами кабельно-проводниковой продукции будут направлены в ближайшее время. Дополнительно прошу Вас направить в наш адрес откорректированную разделительную ведомость по объекту: Обустройство Чаяндинского НГКМ. Куст газовых скважин No2. Приложение: 1. Письмо No [DOCUMENT_NUMBER] от [DATE_TIME], 2. Письмо No [DOCUMENT_NUMBER] от [DATE_TIME]. Исполнительный директор Исполнитель: [PERSON] сот.: [CONTACT] A.P. [PERSON] [ORGANIZATION] [ORGANIZATION] Юридический адрес: [LOCATION] ОГРН [ID], ИНН [ID]. Адрес для корреспонденции: [LOCATION] Тел.: [CONTACT] e-mail: [CONTACT] www.[DOMAIN] На No ОП2-715 от [DATE_TIME] О согласовании стоимости МТР Уважаемый [PERSON]! В ответ на Ваше письмо О согласовании стоимости МТР по объекту: Обустройство Чаяндинского НГКМ. Куст газовых скважин No2 информируем Вас, что Заказчиком проведен анализ стоимости МТР поставки Подрядчика, отсутствующих в составе УЕР и Прейскуранте МТР в рамках договора No [DOCUMENT_NUMBER] от [DATE_TIME] на выполнение строительно-монтажных работ. Информация по предельной цене закупки МТР с учетом доставки до базиса поставки указана в приложении к настоящему письму. Просим обеспечить закуп и поставку МТР на объект строительства по стоимости, не превышающей цены в приложении, при этом право выбора поставщика остается за подрядной организацией. Приложение: 1. Ведомость МТР [DOCUMENT_NUMBER] от [DATE_TIME] на 1 л. С уважением, Руководитель проекта Трубопроводный транспорт ЧНГКМ [PERSON] ДОКУМЕНТ ПОДПИСАН УСИЛЕННОЙ КАВАЛИФИЦИРОВАННОЙ ЭЛЕКТРОННОЙ ПОДПИСЬЮ Сертификат: [ID] Владелец: [PERSON] Действителен: с [DATE_TIME] по [DATE_TIME] [ORGANIZATION] Bx. No [DOCUMENT_NUMBER] от [DATE_TIME] Заказчик: [ORGANIZATION] Номер документа: [DOCUMENT_NUMBER] [ORGANIZATION] Доработ: No [DOCUMENT_NUMBER] от [DATE_TIME] Ведомость МТР [DOCUMENT_NUMBER] от [DATE_TIME] Организация: [ORGANIZATION] Объект: Обычный проект по ремонту и обслуживанию оборудования ХТКМ. Кустра30бхс кважин No2 No n/n Код МТР Наименование МТР (в соответствии с ECM) Единица измерения Стоимость материально-технических ресурсов (руб.) Общая стоимость (руб.) Дата поставки Поставщик  1 2 3 1 770001980569 Кабель Arctic Cab B3-P3БНhr(A)-HF-3ХЛ 34,0Мк-1ТУ [LOCATION]-004-19280226-2022 км [FINANCIAL_DATA] [DATE_TIME] [PERSON] 2 770001980559 Кабель Arctic Cab B3-P3НТhr(A)-HF-3ХЛ 56,0Мк ([CONTACT])-1ТУ [LOCATION]-004-19280226-2022 км [FINANCIAL_DATA] [DATE_TIME] [PERSON] 3 770001980538 Кабель Arctic Cab B3-P3НТhr(A)-HF-3ХЛ 41,5Мк (PE)-1ТУ [LOCATION]-004-19280226-2022 км [FINANCИАЛЬ_DATA] [DATE_TIME] [PERSON] 4 770001980516 Кабель Arctic Cab B3-P3НТhr(A)-HF-3ХЛ 21,5Мк (N)-1ТУ [LOCATION]-004-19280226-2022 км [FINANCIAL_DATA] [DATE_TIME] [PERSON] 5 770001980492 Кабель Arctic Cab INSTALL KMVBhr(A)-LS 51,0МВЭК-ХЛТУ 27.32.13-003-19280226-2019 км [FINANCIAL_DATA] [DATE_TIME] [PERSON] Снабжено: [PERSON] [PERSON] [PERSON] Наименование заказчика: [ORGANIZATION] Номер документа: [DOCUMENT_NUMBER] [ORGANIZATION] Юридический адрес: [LOCATION] ОРГН [ID], ИНН [ID]. Адрес для корреспонденции: [LOCATION] Тел.: [CONTACT] e-mail: [CONTACT] www.[DOMAIN] На No ОП2-711 от [DATE_TIME] Генеральному директору [ORGANIZATION] [PERSON] Уважаемый [PERSON]! В ответ на Ваше письмо О согласовании стоимости МТР по объекту: Транспорт газа на УППГ-4 Чаяндинского НГКМ. Газопровод информируем Вас, что Заказчиком проведен анализ стоимости МТР поставки Подрядчика, отсутствующих в составе УЕР и Прейскуранте МТР в рамках договора No [DOCUMENT_NUMBER] от [DATE_TIME] на выполнение строительно-монтажных работ. Информация по предельной цене закупки МТР с учетом доставки до базиса поставки указана в приложении к настоящему письму. Просим обеспечить закуп и поставку МТР на объект строительства по стоимости, не превышающей цены в приложении, при этом право выбора поставщика остается за подрядной организацией. Приложение: Ведомость МТР [DOCUMENT_NUMBER] от [DATE_TIME] г. на 1 л. С уважением, Руководитель проекта Трубопроводный транспорт Чаяндинского НГКМ [PERSON] [CONTACT] (78865) [CONTACT] [ORGANIZATION] Рег. No [DOCUMENT_NUMBER] от [DATE_TIME] Вх. No [DOCUMENT_NUMBER] от [DATE_TIME]'

translated = MyMemoryTranslator(source='russian', target='french').translate(text)

translated = MyMemoryTranslator(source='french', target='german').translate(translated)

translated = MyMemoryTranslator(source='german', target='russian').translate(translated)

print(translated)

NotValidLength: [ORGANIZATION] Адрес местонахождения: [LOCATION] Тел.: [CONTACT], e-mail: [CONTACT] Адрес местонахождения Обособленного подразделения No1: [LOCATION]; Адрес местонахождения Обособленного подразделения No2: [LOCATION]; Адрес местонахождения Обособленного подразделения No3: [LOCATION]. Тел.: [CONTACT], факс: [CONTACT], ОГРН [ID], ИНН [ID], www.[DOMAIN] Директору по капитальному строительству [ORGANIZATION] [PERSON] Копия: Руководителю проекта Трубопроводный транспорт Чаяндинского НГКМ [ORGANIZATION] [PERSON] [DATE_TIME] О согласовании стоимости МТР Уважаемый [PERSON]! В ответ на Ваши исх. No [DOCUMENT_NUMBER] от [DATE_TIME] и [DOCUMENT_NUMBER] от [DATE_TIME] О согласовании стоимости МТР до базиса поставки УПН Чаяндинского месторождения. В связи с изменением объема согласованной к закупу кабельно-проводниковой продукции, в том числе переноса большего объема в поставку Заказчика, прошу Вас направленные позиции в полном объеме письма NoОП2-715 от [DATE_TIME] и позиции с 1 по

In [19]:
from translate import Translator
translator= Translator(from_lang="russian",to_lang="german")
translation = translator.translate("Привет")
print(translation)

Hallo!


In [42]:
base_text = 'fgdbfbdhdrarg'

exploded_text = [base_text[i:i + 5] for i in range(0, len(base_text), 5)]
exploded_text

['fgdbf', 'bdhdr', 'arg']